In [22]:
import pandas as pd
import requests
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select

%matplotlib inline

In [23]:
df_link=pd.read_csv('cleaned_colorado_sample4.csv')

In [24]:
additional_df = pd.read_csv('cleaned_colorado_sample5.csv')
df_link=df_link.append(additional_df)

In [25]:
#read people csvs in
df_person = pd.read_csv('pData (4).txt',sep='|')
df_person = df_person.append(pd.read_csv('pData (5).txt',sep='|'))
df_person = df_person.append(pd.read_csv('pData (6).txt',sep='|'))
df_person = df_person.append(pd.read_csv('pData (7).txt',sep='|'))
df_person = df_person.append(pd.read_csv('pData (8).txt',sep='|'))
df_person.tail(10)

,full_name,doing_business_as,license_no,license_type_name,license_status_name,addr_line_1,addr_line_4,addr_city,addr_county,addr_state,addr_zipcode,Unnamed: 11
3837,Zachary James Larsen,NaN,M79061,Associated Key - Resident,Approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3838,ZACHARY JAMES MAAS,NaN,M01206,Associated Key - Resident,Withdrawn,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3839,Zachary Steven Moore,NaN,M69426,Associated Key - Resident,Approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3840,Zachary Tyler Mauss,NaN,M25520,Associated Key - Resident,Revoked,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3841,Zachary Wade Nelson,NaN,M71886,Associated Key - Resident,Approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3842,Zane Lowell Mund,NaN,M48558,Associated Key - Resident,Voluntary Surrender,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3843,Zaur Amirgusein Ogly Bairamov,NaN,M58907,Associated Key - Resident,Pending-Voluntary Surrender,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3844,Zechariah Daniel Burrell,NaN,M27430,Associated Key - Resident,Voluntary Surrender,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3845,ZEPHANIAH M WILLIAMS,ZEPHANIAH WILLIAMS,M01618,Associated Key - Resident,Withdrawn,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3846,Zhuoyu Ling,NaN,M75250,Associated Key - Resident,Approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
#clean people csvs
df_person = df_person.drop(columns=df_person.columns[5:])
df_person = df_person.drop(df_person.columns[1],axis=1)

In [27]:
df_person.head(3)

,full_name,license_no,license_type_name,license_status_name
0,Aaron Michael Morris,M77925,Associated Key - Non-Resident,Approved
1,Adam Denmark Cohen,M61175,Associated Key - Non-Resident,Approved
2,Amanda Jeanne Butler,M57073,Associated Key - Non-Resident,Approved


In [28]:
#read facility csvs in 
df_facility = pd.read_csv('Data (4).txt',sep='|')
df_facility = df_facility.append(pd.read_csv('Data (5).txt',sep='|'))
df_facility = df_facility.append(pd.read_csv('Data (6).txt',sep='|'))
df_facility = df_facility.append(pd.read_csv('Data (7).txt',sep='|'))
df_facility = df_facility.append(pd.read_csv('Data (8).txt',sep='|'))
df_facility = df_facility.append(pd.read_csv('Data (9).txt',sep='|'))
df_facility = df_facility.append(pd.read_csv('Data (10).txt',sep='|'))
df_facility.tail(10)

,full_name,doing_business_as,license_no,license_type_name,license_status_name,addr_line_1,addr_line_4,addr_city,addr_county,addr_state,addr_zipcode,Unnamed: 11
1264,WINPARK I LLC,THE HERBAN CENTER,403-00329,Optional Premises,Withdrawn,3131 Hwy 7,Estes Park CO 80517,Estes Park,Larimer,CO,80517,NaN
1265,WINPARK I LLC,THE HERBAN CENTER,403-01225,Optional Premises,Withdrawn,4010 Clear View Frontage Rd #101A,Colorado Springs CO 80911,Colorado Springs,El Paso,CO,80911,NaN
1266,WISE CANNABIS CO LLC,NaN,404R-00193,Retail Marijuana Products Mfg,Withdrawn,21950 Highway 285,Fairplay CO 80440,Fairplay,Park,CO,80440,NaN
1267,WORLD APOTHIC LLC,NaN,403R-00923,Retail Marijuana Cultivation Facility,Withdrawn,3630 County Road 650,Rye CO 81069,Rye,Pueblo,CO,81069,NaN
1268,WTJ FAMILY ENTERPRISES LLC,WTJ FAMILY ENTERPRISES LLC,404-00059,Infused Product Manufacturer,Withdrawn,319 Mount View,Colorado Springs CO 80907,Colorado Springs,El Paso,CO,80907,NaN
1269,YEOMANSAX LLC,THE HEALTH CENTER,402R-00328,Retail Marijuana Store,Withdrawn,12600 East Smith Road,Aurora CO 80011,Aurora,Adams,CO,80011,NaN
1270,YUMA WAY LLC,CHERRY PEAK,402-01299,Center - Type 1,Withdrawn,4601 East Mississippi Avenue,Glendale CO 80246,Glendale,Arapahoe,CO,80246,NaN
1271,ZAPOTK CORPORATION LTD,NaN,403R-00781,Retail Marijuana Cultivation Facility,Withdrawn,778 County Road 230,Walsenburg CO 81089,Walsenburg,Huerfano,CO,81089,NaN
1272,ZEN FARMACY LLC,THE ZEN FARMACY LLC,402-00668,Center - Type 1,Withdrawn,323 3RD AVE,Longmont CO 80501,Longmont,Boulder,CO,80501,NaN
1273,ZUNI WELLNESS CENTER LLC,THE CLINIC ALTERNATIVE MEDICINE DBA THE CLINIC,402-00136,Center - Type 1,Withdrawn,1045 ZUNI ST,Denver CO 80204,Denver,Denver,CO,80204,NaN


In [29]:
#clean facility csv
df_facility = df_facility.drop('Unnamed: 11',axis=1)

In [8]:
#make csvs of df_facility and df_person
df_facility.to_csv('colorado_facility.csv',index=False)
df_person.to_csv('colorado_person.csv',index=False)

In [30]:
#taking a look at the data by county
df_facility.addr_county.value_counts()

Denver         2071
El Paso         666
Pueblo          509
Boulder         489
Adams           218
Garfield        183
Larimer         166
Arapahoe        134
Jefferson       119
Saguache        118
Las Animas       97
Mesa             88
La Plata         86
Clear Creek      69
Park             66
Routt            62
Weld             60
Pitkin           60
Fremont          56
Eagle            51
Summit           44
San Miguel       41
Huerfano         40
Montezuma        37
Gunnison         36
Chaffee          36
Gilpin           28
Ouray            27
Archuleta        27
Crowley          24
Morgan           23
Grand            21
Costilla         21
Lake             17
Teller           17
Douglas          12
San Juan         11
Delta            11
Montrose          8
Otero             7
Conejos           7
Elbert            5
Moffat            5
Alamosa           4
Sedgwick          4
Hinsdale          3
Bent              2
Washington        1
Custer            1
Yuma              1


# Investigating if the Data is Join-able
https://demography.dola.colorado.gov/census-acs/american-community-survey-data/#american-community-survey-data-for-colorado

In [31]:
#do the counties line up between census data and the department of revenue?
income_df=pd.read_csv('colorado_income.csv',sep='|')
income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 4 columns):
FIPS                     65 non-null int64
GeographicArea           65 non-null object
MedianHouseholdIncome    65 non-null int64
MarginofError(+/-)       65 non-null int64
dtypes: int64(3), object(1)
memory usage: 2.1+ KB


In [32]:
merged_df=df_facility.merge(income_df, left_on='addr_county',right_on='GeographicArea',how='left')

In [33]:
#figure out the counties that didn't match. If everything matched the returning list will be empty
merged_df[merged_df['GeographicArea'].isnull()].addr_county.value_counts()

Series([], Name: addr_county, dtype: int64)

In [34]:
#Looks like we have to get rid of the spaces for the merge (compare the below frame)
# income_df.GeographicArea.value_counts()

# Checking that All the People and Facilities Are There
Adding this section to do when/if I ever totally download all the data and my scraper doesn't crash

In [60]:
#count the number of unique facilities I scraped
df_link.flicense.nunique()

404-00591     1042
402-00194      807
404-00037      567
403R-00173     466
404R-00128     428
402R-00085     412
403-01717      412
403R-00202     389
402-00966      314
402-00080      274
403R-00434     273
403R-00276     270
404R-00269     267
404R-00383     230
402-01072      211
403R-01094     209
403R-00685     194
404R-00404     194
403R-00790     186
402-00919      178
404R-00329     167
403R-01148     164
402R-00584     162
404-00596      149
403-01740      147
403-01895      145
402-00732      142
403R-00966     139
402-01015      137
403R-00839     132
              ... 
402-00338        1
404-00187        1
402-00796        1
404-00088        1
404-00017        1
403R-00770       1
403R-00865       1
16-03190         1
403R-00837       1
402R-00731       1
402-00163        1
402-00634        1
404-00254        1
402R-00413       1
402-00661        1
404R-00393       1
404R-00249       1
13-00116         1
403-00831        1
402-00845        1
11-00029         1
402-00904   

In [61]:
#compared to number of unique facilities in the facility list...where is this difference coming from?
df_facility.license_no.nunique()

403R-00771    1
402-00264     1
403R-00260    1
404R-00358    1
404R-00327    1
403-01721     1
402-00148     1
403R-00104    1
402R-00693    1
404R-00136    1
404R-00211    1
403-00304     1
403-01542     1
404-00105     1
402R-00295    1
403-00345     1
402R-00507    1
403-00349     1
403-01838     1
402R-00434    1
402-00510     1
403-00659     1
402R-00498    1
403-00523     1
403-00159     1
402R-00228    1
402-00818     1
403R-00304    1
403-00960     1
402-00036     1
             ..
402-00498     1
403-01901     1
403R-00118    1
404R-00199    1
402R-00103    1
403R-00868    1
403-00396     1
403R-00097    1
402R-00148    1
403-00064     1
403R-00370    1
403-01037     1
402-00969     1
405R-00006    1
402R-00422    1
404R-00367    1
403-00855     1
404R-00177    1
403-01317     1
403-00572     1
402-01001     1
402R-00653    1
403-01361     1
402-00207     1
404R-00293    1
403R-00323    1
403-01523     1
402R-00210    1
403-01086     1
403R-00786    1
Name: license_no, Length

In [36]:
df_person[df_person['license_status_name']=='Approved'].license_no.nunique()

27395

In [38]:
df_facility.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5968 entries, 0 to 1273
Data columns (total 11 columns):
full_name              5968 non-null object
doing_business_as      4396 non-null object
license_no             5968 non-null object
license_type_name      5968 non-null object
license_status_name    5968 non-null object
addr_line_1            5965 non-null object
addr_line_4            5968 non-null object
addr_city              5968 non-null object
addr_county            5890 non-null object
addr_state             5968 non-null object
addr_zipcode           5968 non-null int64
dtypes: int64(1), object(10)
memory usage: 559.5+ KB


# Counting
For every county, I want to know how many facilities there are

In [39]:
county_df = merged_df.GeographicArea.value_counts().rename_axis('county').reset_index(name='facility_counts')

In [40]:
county_df

,county,facility_counts
0,Denver,2071
1,El Paso,666
2,Pueblo,509
3,Boulder,489
4,Adams,218
5,Garfield,183
6,Larimer,166
7,Arapahoe,134
8,Jefferson,119
9,Saguache,118


In [41]:
#merge this with the income data for my dataframe
county_income_df=county_df.merge(income_df,left_on='county',right_on='GeographicArea',how='left')

In [42]:
#I don't think I need the FIPS column. Will keep MarginofError column because that could be interesting?
county_income_df=county_income_df.drop(columns='FIPS')

In [43]:
#export to csv to read
county_income_df.to_csv('county_income.csv',index=False)

It would also be cool to just see who the top 10 owners are and where their facilities are!

In [44]:
counting_peeps= df_link.merge(df_person,left_on = 'plicense',right_on='license_no',how='left')

In [56]:
counting_peeps[['plicense','flicense','full_name_x']].full_name_x.value_counts().head(10)

Jonathan Robert Boord      801
John Joseph Fritzel        537
Winston Harold Conkling    410
Ethan Daniel Borg          314
Adam Denmark Cohen         272
Matthew Thomas Aiken       265
Marcus John Fauth          242
Steven Hamilton Brooks     209
Scott Alan Embree, Jr      202
Allyson Danielle Feiler    193
Name: full_name_x, dtype: int64

In [55]:
counting_peeps[counting_peeps['full_name_x']=='Jonathan Robert Boord']

,flicense,full_name_x,plicense,full_name_y,license_no,license_type_name,license_status_name
231,402-00194,Jonathan Robert Boord,M10239,Jonathan Robert Boord,M10239,Associated Key - Resident,Approved
6042,402-00194,Jonathan Robert Boord,M10239,Jonathan Robert Boord,M10239,Associated Key - Resident,Approved
6043,402-00194,Jonathan Robert Boord,M10239,Jonathan Robert Boord,M10239,Associated Key - Resident,Approved
6044,402-00194,Jonathan Robert Boord,M10239,Jonathan Robert Boord,M10239,Associated Key - Resident,Approved
6045,402-00194,Jonathan Robert Boord,M10239,Jonathan Robert Boord,M10239,Associated Key - Resident,Approved
6046,402-00194,Jonathan Robert Boord,M10239,Jonathan Robert Boord,M10239,Associated Key - Resident,Approved
6047,402-00194,Jonathan Robert Boord,M10239,Jonathan Robert Boord,M10239,Associated Key - Resident,Approved
6048,402-00194,Jonathan Robert Boord,M10239,Jonathan Robert Boord,M10239,Associated Key - Resident,Approved
6049,402-00194,Jonathan Robert Boord,M10239,Jonathan Robert Boord,M10239,Associated Key - Resident,Approved
6050,402-00194,Jonathan Robert Boord,M10239,Jonathan Robert Boord,M10239,Associated Key - Resident,Approved
